# Python数据分析建模--分类与预测

* 日期：2017/02/16

## 项目简介
* [项目地址](https://www.kaggle.com/uciml/glass)
* 任务：根据玻璃材料的9个属性对未知的玻璃类别进行分类和预测

# 1. 准备阶段
## 加载相关模块和库

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

%matplotlib inline

## 加载并查看数据

In [ ]:
dataset_path = '../input/glass.csv'
data = pd.read_csv(dataset_path)
feature_names = data.columns[:-1].tolist()
print('数据形状：', data.shape)
print('共有{}条记录'.format(data.shape[0]))
print('共有'+ str(data.shape[0]) +'条记录')
print('共有{}个特征：{}'.format(len(feature_names), feature_names))

print()
print('数据预览：')
print(data.head())

print('属性类型：')
print(data.dtypes)

# 2. 数据分析及可视化
## 查看统计信息

In [ ]:
print('数据统计信息：')
data.describe()

通过简单的查看，发现特征数据的范围不一致，对模型的学习过程有一定影响。比如RI和Si的均值相差过大，不在同一个数值范围中。

接下来查看各类别的数据记录个数。

In [ ]:
data['Type'].value_counts()

类别分布不平衡，其中类别1和2占了大多数。

## 数据可视化
查看特征的数据分布情况

偏度(skewness)
* [Wikipedia](https://zh.wikipedia.org/wiki/%E5%81%8F%E5%BA%A6)
* [百度百科](http://baike.baidu.com/item/%E5%81%8F%E5%BA%A6/8626571?fr=aladdin)

例子：
![偏度值为正](../input/test111/pos_skew.png) 例如：工资数据。一个公司中许多员工得到的工资相对较少，而越来越少的人得到非常高的工资。
![偏度值为负](./neg_skew.png) 例如：灯泡。极少量的灯泡会立即烧坏，大多数灯泡会持续相当长的时间。

In [ ]:
for feature in feature_names:
    skew = data[feature].skew()
    sns.distplot(data[feature], label='skew = %.3f' %(skew))
    plt.legend(loc='best')
    plt.show()

In [ ]:
sns.boxplot(data[feature_names])
plt.show()

多变量数据可视化

In [ ]:
plt.figure(figsize=(8, 8))
sns.pairplot(data[feature_names])
plt.show()

# 3. 数据处理
## 分割数据集

In [ ]:
X = data[feature_names].values
y = data['Type'].values

# 随机数生成种子
seed = 5
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size , random_state = seed)

# 4. 数据建模

In [ ]:
# 选择模型，交叉验证
k_range = range(1, 31)
cv_scores = []
print('交叉验证：')
for k in k_range:
    knn = KNeighborsClassifier(k)
    scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
    score_mean = scores.mean()
    cv_scores.append(score_mean)
    print('%i: %.4f' % (k, score_mean))

best_k = np.argmax(cv_scores) + 1
print('最优K: ', best_k)

plt.plot(k_range, cv_scores)
plt.xlabel('K')
plt.ylabel('Accuracy')
plt.show()

# 5. 分类和预测
* knn模型
![knn](./knn.png)

In [ ]:
# 训练模型
knn_model = KNeighborsClassifier(best_k)
knn_model.fit(X_train, y_train)
print('测试模型，准确率：', knn_model.score(X_test, y_test))

# 6. 思考：如何提升准确率？

* a. 非平衡样本数据处理
* b. 特征归一化
* c. 尝试其他分类预测模型
* d. 尝试获得更多数据